In [ ]:
import numpy as np 
import pandas as pd
import os

In [3]:
os.makedirs('datasets/prcc_proprecess')
os.makedirs('datasets/prcc_proprecess/train')
os.makedirs('datasets/prcc_proprecess/query')
os.makedirs('datasets/prcc_proprecess/gallery')
os.makedirs('datasets/prcc_proprecess/train_sketch')
os.makedirs('datasets/prcc_proprecess/query_sketch')
os.makedirs('datasets/prcc_proprecess/gallery_sketch')

In [4]:
from fastai.vision.all import get_image_files
import shutil
train_set=set()

for root, dirs, files in os.walk("/kaggle/input/prccdata1/prcc/rgb/train"):
  for dir in dirs:
    train_set.add(dir)
    path=os.path.join(root,dir)
    image_names=get_image_files(path)
    for i,name in enumerate(image_names):
       name=str(name)
       last_name=name.split('/')[-1]
       cam=last_name[0]
       if cam=='A':
            cam=0
       elif cam=='B':
            cam=1
       elif cam=='C':
            cam=2
       
       last_name=last_name.split('.')[0][-3:]
       new_name=os.path.join('datasets','prcc_proprecess','train',dir+'_'+str(cam)+'_'+last_name+'.jpg')
       shutil.copy(str(name),new_name)
  break




for root, dirs, files in os.walk("/kaggle/input/prccdata1/prcc/sketch/train"):
  for dir in dirs:
    train_set.add(dir)
    path=os.path.join(root,dir)
    image_names=get_image_files(path)

    for i,name in enumerate(image_names):
       name=str(name)
       last_name=name.split('/')[-1]
       cam=last_name[0]
       if cam=='A':
            cam=0
       elif cam=='B':
            cam=1
       elif cam=='C':
            cam=2
       last_name=last_name.split('.')[0][-3:]
       new_name=os.path.join('datasets','prcc_proprecess','train_sketch',dir+'_'+str(cam)+'_'+last_name+'.jpg')
       shutil.copy(str(name),new_name)
  break


for root, dirs, files in os.walk("/kaggle/input/prccdata1/prcc/rgb/val"):
  for dir in dirs:
    train_set.add(dir)
    path=os.path.join(root,dir)
    image_names=get_image_files(path)
    A_query=True
    B_query=True
    C_query=True
    for i,name in enumerate(image_names):
      name=str(name)
      last_name=name.split('/')[-1]
      
      if last_name[0]=='A': 
           if A_query:

               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','query',dir+'_0'+'_'+last_name+'.jpg')

               A_query=False
           else:
               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','gallery',dir+'_0'+'_'+last_name+'.jpg')
      
      elif last_name[0]=='B': 
           if B_query:

               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','query',dir+'_1'+'_'+last_name+'.jpg')
               B_query=False
           else:
               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','gallery',dir+'_1'+'_'+last_name+'.jpg')
     
      elif last_name[0]=='C' :
           if C_query:

               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','query',dir+'_2'+'_'+last_name+'.jpg')
               C_query=False
           else:
               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','gallery',dir+'_2'+'_'+last_name+'.jpg')
      shutil.copy(name,new_name)
      
              
  break

for root, dirs, files in os.walk("/kaggle/input/prccdata1/prcc/sketch/val"):
  for dir in dirs:
    train_set.add(dir)
    path=os.path.join(root,dir)
    image_names=get_image_files(path)
    A_query=True
    B_query=True
    C_query=True
    for i,name in enumerate(image_names):
      name=str(name)
      last_name=name.split('/')[-1]
      
      if last_name[0]=='A':
           if A_query:

               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','query_sketch',dir+'_0'+'_'+last_name+'.jpg')
               A_query=False
           else:
               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','gallery_sketch',dir+'_0'+'_'+last_name+'.jpg')
      
      elif last_name[0]=='B': 
           if B_query:

               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','query_sketch',dir+'_1'+'_'+last_name+'.jpg')
               B_query=False
           else:
               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','gallery_sketch',dir+'_1'+'_'+last_name+'.jpg')

      elif last_name[0]=='C': 
           if C_query:

               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','query_sketch',dir+'_2'+'_'+last_name+'.jpg')
               C_query=False
           else:
               last_name=last_name.split('.')[0][-3:]
               new_name=os.path.join('datasets','prcc_proprecess','gallery_sketch',dir+'_2'+'_'+last_name+'.jpg')
      shutil.copy(name,new_name)
      
  break
print(train_set)
train_set=list(train_set)
print(train_set)
train_label={name:i for i,name in enumerate(train_set)}


In [6]:
import torchvision.transforms as T
def build_change_transforms(cfg, training=True):
    normalize_transform = T.Normalize(mean=cfg['mean'], std=cfg['std'])
    if training:
        transform = T.Compose([
            T.Resize(cfg['train_size']),
            T.ColorJitter(),
            T.RandomGrayscale(p=0.05),
            T.ToTensor(),
            normalize_transform,
        ])
    else:
        transform = T.Compose([
            T.Resize(cfg['test_size']),
            T.ToTensor(),
            normalize_transform
        ])

    return transform


import albumentations as A
def build_transform(cfg,training=True):
  if training:
    transform=A.Compose([
        A.Resize(228,228),
        A.RandomBrightnessContrast(),
        A.HorizontalFlip(p=0.5),
        A.Normalize()
    ])
  else:
    transform=A.Compose([
        A.Resize(228,228),
        A.Normalize()
    ])
  return transform
from fastai.vision.all import get_image_files
import torch.utils.data as Data
import json
import os.path as osp

class PRCC:
  def __init__(self,root):
    self.root=root
    self.train_dir=osp.join(self.root,'train')
    self.query_dir=osp.join(self.root,'query')
    self.gallery_dir=osp.join(self.root,'gallery')
    self.train_sketch_dir=osp.join(self.root,'train_sketch')
    self.query_sketch_dir=osp.join(self.root,'query_sketch')
    self.gallery_sketch_dir=osp.join(self.root,'gallery_sketch')
    
    
    
    train=self._process_dir(self.train_dir)
    query=self._process_dir(self.query_dir)
    gallery=self._process_dir(self.gallery_dir)
    train_sketch=self._process_dir(self.train_sketch_dir)
    query_sketch=self._process_dir(self.query_sketch_dir)
    gallery_sketch=self._process_dir(self.gallery_sketch_dir)


    self.train=train
    self.query=query
    self.gallery=gallery
    self.train_sketch=train_sketch
    self.query_sketch=query_sketch
    self.gallery_sketch=gallery_sketch
    self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
    self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
    self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)

  
  def get_imagedata_info(self,data):
    pids,cams=[],[]
    for _,pid,camid in data:
      pids+=[pid]
      cams+=[camid]
    pids=set(pids)
    cams=set(cams)
    num_pids=len(pids)
    num_cams=len(cams)
    num_imgs=len(data)
    return num_pids,num_imgs,num_cams

  def _process_dir(self,dir_path):
    image_names=get_image_files(dir_path)
    dataset=[]
    pid_set=set()
    for name in image_names:
      name=str(name)
      path=name.split('/')[-1]
      pid,cam,_=path.split('_')
      pid=train_label[pid]
      dataset.append((name,pid,cam))
    return dataset


In [7]:
import copy
import random
import torch
from collections import defaultdict

import numpy as np
from torch.utils.data.sampler import Sampler
class DateRandomIdentitySampler(Sampler):
    """
    Randomly sample N identities, then for each identity,
    randomly sample K instances, therefore batch size is N*K.
    Args:
    - data_source (list): list of (img_path, pid, camid).
    - num_instances (int): number of instances per identity in a batch.
    - batch_size (int): number of examples in a batch.
    """

    def __init__(self, data_source, batch_size, num_instances):
        self.data_source = data_source
        self.batch_size = batch_size
        self.num_instances = num_instances
        self.num_pids_per_batch = self.batch_size // self.num_instances
        self.index_dic = defaultdict(list)
        for index, (_, pid, _) in enumerate(self.data_source):
            self.index_dic[pid].append(index)
        self.pids = list(self.index_dic.keys())

        # estimate number of examples in an epoch
        self.length = 0
        for pid in self.pids:
            idxs = self.index_dic[pid]
            num = len(idxs)
            if num < self.num_instances:
                num = self.num_instances
                
            self.length += num - num % self.num_instances

    def __iter__(self):
        batch_idxs_dict = defaultdict(list)

        for pid in self.pids:
            idxs = copy.deepcopy(self.index_dic[pid])
            if len(idxs) < self.num_instances:
                idxs = np.random.choice(idxs, size=self.num_instances, replace=True)
            random.shuffle(idxs)
            batch_idxs = []
            for idx in idxs:
                batch_idxs.append(idx)
                if len(batch_idxs) == self.num_instances:
                    batch_idxs_dict[pid].append(batch_idxs)
                    batch_idxs = []

        avai_pids = copy.deepcopy(self.pids)
        final_idxs = []

        while len(avai_pids) >= self.num_pids_per_batch:
            selected_pids = random.sample(avai_pids, self.num_pids_per_batch)
            for pid in selected_pids:
                batch_idxs = batch_idxs_dict[pid].pop(0)
                final_idxs.extend(batch_idxs)
                if len(batch_idxs_dict[pid]) == 0:
                    avai_pids.remove(pid)

        self.length = len(final_idxs)
        return iter(final_idxs)

    def __len__(self):
        return self.length


In [8]:
from PIL import Image

import torch.utils.data as Data
class ImageDataset(Data.Dataset):
    def __init__(self, dataset,data,transform=None):
        self.dataset = dataset
        self.transform = transform
        self.data=data

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path, pid, camid = self.dataset[index]
        img = Image.open(img_path)

        path=img_path.split('/')[-1]
        data_type=img_path.split('/')[-2]
        if data_type.startswith('train'):
          sketch_dir=self.data.train_sketch_dir
        elif data_type.startswith('query'):
          sketch_dir=self.data.query_sketch_dir
        else:
          sketch_dir=self.data.gallery_sketch_dir
        sketch_path=os.path.join(sketch_dir,path)
        sketch=Image.open(sketch_path)
        image=np.array(img)
        
        sketch=np.array(sketch)
       
        transform=self.transform(image=image,mask=sketch)
        image=transform['image']
        sketch=transform['mask']
        
        image=torch.from_numpy(image.transpose(2,0,1))
        sketch=torch.from_numpy(sketch).unsqueeze(0)
        sketch=sketch.repeat(3,1,1)
        try:
          if int(camid)==0 or int(camid)==1:
            date=0
          else:
            date=1
        except:
          print(camid)
        
        return image,sketch,int(pid), int(camid),date






def train_collate_fn1(batch):
    imgs, sketch,pids,camid,date = zip(*batch)
    pids = torch.tensor(pids, dtype=torch.int64)
    return torch.stack(imgs, dim=0),torch.stack(sketch,dim=0), pids

###对于验证集而言，为了提高验证的真实性，我们应该防止同一摄像头的图片进入验证（同一摄像头相当于数据泄露）
def val_collate_fn1(batch):
    imgs,sketch,pids, camids,date = zip(*batch)
    return torch.stack(imgs, dim=0),torch.stack(sketch,dim=0),pids, camids,date



def make_dataloader(dataset,cfg):
    train_transform=build_transform(cfg,training=True)
    test_transform=build_transform(cfg,training=False)
    
   
    data=dataset
    num_classes=data.num_train_pids
    train_set=ImageDataset(data.train,data,train_transform)

    train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     sampler=DateRandomIdentitySampler(data.train,
                                    cfg['train_bs'],cfg['train_K_instances']),
                                    num_workers=cfg['num_workers'],
                                    collate_fn=train_collate_fn1)
    val_set = ImageDataset(data.query + data.gallery,data,test_transform)
    val_loader = Data.DataLoader(
        val_set, batch_size=cfg['test_bs'], shuffle=False, num_workers=cfg['num_workers'],
        collate_fn=val_collate_fn1
    )
    return train_loader, val_loader, len(data.query), num_classes


In [9]:
import torch.nn as nn
def normalize(x, axis=-1):
    """Normalizing to unit length along the specified dimension.
    Args:
      x: pytorch Variable
    Returns:
      x: pytorch Variable, same shape as input
    """
    x = 1. * x / (torch.norm(x, 2, axis, keepdim=True).expand_as(x) + 1e-12)
    return x


def euclidean_dist(x, y):
    """ 
    Args:
      x: pytorch Variable, with shape [m, d]
      y: pytorch Variable, with shape [n, d]
    Returns:
      dist: pytorch Variable, with shape [m, n]
    """
    m, n = x.size(0), y.size(0)
    xx = torch.pow(x, 2).sum(1, keepdim=True).expand(m, n)
    yy = torch.pow(y, 2).sum(1, keepdim=True).expand(n, m).t()
    dist = xx + yy
    dist.addmm_(1, -2, x, y.t())
    dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
    return dist


def hard_example_mining(dist_mat, labels, return_inds=False):
    """For each anchor, find the hardest positive and negative sample.
    Args:
      dist_mat: pytorch Variable, pair wise distance between samples, shape [N, N]
      labels: pytorch LongTensor, with shape [N]
      return_inds: whether to return the indices. Save time if `False`(?)
    Returns:
      dist_ap: pytorch Variable, distance(anchor, positive); shape [N]
      dist_an: pytorch Variable, distance(anchor, negative); shape [N]
      p_inds: pytorch LongTensor, with shape [N];
        indices of selected hard positive samples; 0 <= p_inds[i] <= N - 1
      n_inds: pytorch LongTensor, with shape [N];
        indices of selected hard negative samples; 0 <= n_inds[i] <= N - 1
    NOTE: Only consider the case in which all labels have same num of samples,
      thus we can cope with all anchors in parallel.
    """

    assert len(dist_mat.size()) == 2
    assert dist_mat.size(0) == dist_mat.size(1)
    N = dist_mat.size(0)

    # shape [N, N]
    is_pos = labels.expand(N, N).eq(labels.expand(N, N).t())
    is_neg = labels.expand(N, N).ne(labels.expand(N, N).t())

    # `dist_ap` means distance(anchor, positive)
    # both `dist_ap` and `relative_p_inds` with shape [N, 1]
    dist_ap, relative_p_inds = torch.max(
        dist_mat[is_pos].contiguous().view(N, -1), 1, keepdim=True)
    # `dist_an` means distance(anchor, negative)
    # both `dist_an` and `relative_n_inds` with shape [N, 1]
    dist_an, relative_n_inds = torch.min(
        dist_mat[is_neg].contiguous().view(N, -1), 1, keepdim=True)
    # shape [N]
    dist_ap = dist_ap.squeeze(1)
    dist_an = dist_an.squeeze(1)

    if return_inds:
        # shape [N, N]
        ind = (labels.new().resize_as_(labels)
               .copy_(torch.arange(0, N).long())
               .unsqueeze(0).expand(N, N))
        # shape [N, 1]
        p_inds = torch.gather(
            ind[is_pos].contiguous().view(N, -1), 1, relative_p_inds.data)
        n_inds = torch.gather(
            ind[is_neg].contiguous().view(N, -1), 1, relative_n_inds.data)
        # shape [N]
        p_inds = p_inds.squeeze(1)
        n_inds = n_inds.squeeze(1)
        return dist_ap, dist_an, p_inds, n_inds

    return dist_ap, dist_an


class TripletLoss(object):
    """Modified from Tong Xiao's open-reid (https://github.com/Cysu/open-reid).
    Related Triplet Loss theory can be found in paper 'In Defense of the Triplet
    Loss for Person Re-Identification'."""

    def __init__(self, margin=None):
        self.margin = margin
        if margin is not None:
            self.ranking_loss = nn.MarginRankingLoss(margin=margin)
        else:
            self.ranking_loss = nn.SoftMarginLoss()

    def __call__(self, global_feat, labels, normalize_feature=True):
        if normalize_feature:
            global_feat = normalize(global_feat, axis=-1)
        dist_mat = euclidean_dist(global_feat, global_feat)
        dist_ap, dist_an = hard_example_mining(
            dist_mat, labels)
        y = dist_an.new().resize_as_(dist_an).fill_(1)
        if self.margin is not None:
            loss = self.ranking_loss(dist_an, dist_ap, y)
        else:
            loss = self.ranking_loss(dist_an - dist_ap, y)
        return loss

class CrossEntropyLabelSmooth(nn.Module):
    """Cross entropy loss with label smoothing regularizer.
    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.
    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """
    def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.use_gpu = use_gpu
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).data.cpu(), 1)
        if self.use_gpu: targets = targets.cuda()
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (- targets * log_probs).mean(0).sum()
        return loss

from typing import Tuple
from torch import Tensor
from bisect import bisect_right
def convert_label_to_similarity(normed_feature: Tensor, label: Tensor) -> Tuple[Tensor, Tensor]:
    similarity_matrix = normed_feature @ normed_feature.transpose(1, 0)
    label_matrix = label.unsqueeze(1) == label.unsqueeze(0)

    positive_matrix = label_matrix.triu(diagonal=1)
    negative_matrix = label_matrix.logical_not().triu(diagonal=1)

    similarity_matrix = similarity_matrix.view(-1)
    positive_matrix = positive_matrix.view(-1)
    negative_matrix = negative_matrix.view(-1)
    return similarity_matrix[positive_matrix], similarity_matrix[negative_matrix]

class CircleLoss(nn.Module):
    def __init__(self, m: float, gamma: float) -> None:
        super(CircleLoss, self).__init__()
        self.m = m
        self.gamma = gamma
        self.soft_plus = nn.Softplus()


    def forward(self,feat,label) -> Tensor:

        feat=normalize(feat,axis=-1)
        sp, sn = convert_label_to_similarity(feat, label)
        ap = torch.clamp_min(- sp.detach() + 1 + self.m, min=0.)
        an = torch.clamp_min(sn.detach() + self.m, min=0.)

        delta_p = 1 - self.m
        delta_n = self.m

        logit_p = - ap * (sp - delta_p) * self.gamma
        logit_n = an * (sn - delta_n) * self.gamma

        loss = self.soft_plus(torch.logsumexp(logit_n, dim=0) + torch.logsumexp(logit_p, dim=0))

        return loss
class WarmupMultiStepLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
            self,
            optimizer,
            milestones,
            gamma=0.1,
            warmup_factor=1.0 / 3,
            warmup_iters=500,
            warmup_method="linear",
            last_epoch=-1,
    ):
        if not list(milestones) == sorted(milestones):
            raise ValueError(
                "Milestones should be a list of" " increasing integers. Got {}",
                milestones,
            )

        if warmup_method not in ("constant", "linear"):
            raise ValueError(
                "Only 'constant' or 'linear' warmup_method accepted"
                "got {}".format(warmup_method)
            )
        self.milestones = milestones
        self.gamma = gamma
        self.warmup_factor = warmup_factor
        self.warmup_iters = warmup_iters
        self.warmup_method = warmup_method
        super(WarmupMultiStepLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        warmup_factor = 1
        if self.last_epoch < self.warmup_iters:
            if self.warmup_method == "constant":
                warmup_factor = self.warmup_factor
            elif self.warmup_method == "linear":
                alpha = self.last_epoch / self.warmup_iters
                warmup_factor = self.warmup_factor * (1 - alpha) + alpha
        return [
            base_lr
            * warmup_factor
            * self.gamma ** bisect_right(self.milestones, self.last_epoch)
            for base_lr in self.base_lrs
        ]
class Flat(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, cfg,start_lr,anneal_start=0.6,t=4, last_epoch=-1):
        self.epochs = cfg['epochs']
        self.start = anneal_start
        self.t=t
        self.start_lr=start_lr


        super(Flat, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < int(self.epochs * self.start):
            return [base_lr for base_lr in self.base_lrs]
        else:
            return [
                1e-5+(self.start_lr-1e-5)*(1+math.cos(math.pi*(self.last_epoch-int(self.epochs*self.start))/t))/2
                for base_lr in self.base_lrs
            ]


In [11]:
class GeneralizedMeanPooling(nn.Module):
    r"""Applies a 2D power-average adaptive pooling over an input signal composed of several input planes.
    The function computed is: :math:`f(X) = pow(sum(pow(X, p)), 1/p)`
        - At p = infinity, one gets Max Pooling
        - At p = 1, one gets Average Pooling
    The output is of size H x W, for any input size.
    The number of output features is equal to the number of input planes.
    Args:
        output_size: the target output size of the image of the form H x W.
                     Can be a tuple (H, W) or a single H for a square image H x H
                     H and W can be either a ``int``, or ``None`` which means the size will
                     be the same as that of the input.
    """

    def __init__(self, norm, output_size=1, eps=1e-6):
        super(GeneralizedMeanPooling, self).__init__()
        assert norm > 0
        self.p = float(norm)
        self.output_size = output_size
        self.eps = eps

    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        return torch.nn.functional.adaptive_avg_pool2d(x, self.output_size).pow(1. / self.p)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + str(self.p) + ', ' \
               + 'output_size=' + str(self.output_size) + ')'


class GeneralizedMeanPoolingP(GeneralizedMeanPooling):
    """ Same, but norm is trainable
    """

    def __init__(self, norm=3, output_size=1, eps=1e-6):
        super(GeneralizedMeanPoolingP, self).__init__(norm, output_size, eps)
        self.p = nn.Parameter(torch.ones(1) * norm)


In [12]:
from torchvision.models import resnet50
from geffnet import efficientnet_b0, efficientnet_b1
def build_backbone(model_name):
    if model_name=='resnet50':
        model = resnet50(True)
        stride = 1
        model.layer4[0].downsample[0].stride = stride
        model.layer4[0].conv2.stride = stride
        base = nn.Sequential(
            model.conv1,
            model.bn1,
            model.maxpool,
            model.layer1,
            model.layer2,
            model.layer3,
            model.layer4
        )
        feat=2048
    elif model_name == 'eff_b0':
        backbone = efficientnet_b0(True)
        backbone = nn.Sequential(
            backbone.conv_stem,
            backbone.bn1,
            backbone.act1,
            backbone.blocks,
            backbone.conv_head,
            backbone.bn2,
            backbone.act2,
            nn.Conv2d(1280, 2048, 1)
        )
        base = backbone
        feat = 2048
    elif model_name == 'eff_b1':
        backbone = efficientnet_b1(True)
        backbone = nn.Sequential(
            backbone.conv_stem,
            backbone.bn1,
            backbone.act1,
            backbone.blocks,
            backbone.conv_head,
            backbone.bn2,
            backbone.act2,
            nn.Conv2d(1280, 2048, 1)
        )
        base = backbone
        feat = 2048
    return base,feat

In [58]:
import torch.nn as nn
import math
import torch
import torch.nn.functional as F
class Mask_Branch(nn.Module):
    def __init__(self,cfg,num_class):
        super(Mask_Branch, self).__init__()
        self.cls_criterion=CrossEntropyLabelSmooth(num_class)
        self.cls_criterion.to('cuda:0')
        self.circle_criterion=CircleLoss(m=0.25,gamma=80)
        self.circle_criterion.to('cuda:0')
        self.N=cfg['N']
        self.M=cfg['M']
        self.lambdak=nn.Parameter(torch.from_numpy(np.array([0.05]*self.N)))
        self.a0=-1*math.pi
        self.b0=1*math.pi
        self.a1=-1*math.pi/4
        self.b1=-3*math.pi/4
        self.a2=3*math.pi/4
        self.b2=math.pi/4
        self.a3 = 5*math.pi/8
        self.b3 = 3*math.pi/8
        self.lambdad = nn.Parameter(torch.from_numpy(np.array([0.05]*self.M)))
        self.R_min=0 
        self.R_max=2
        self.backbone_list=nn.ModuleList()
        for i in range(4):
            backbone,feat_dim=build_backbone(cfg['backbone'])
            self.backbone_list.append(backbone)

        self.pool_list=nn.ModuleList()
        for i in range(4):
            for j in range(4):
                pool=GeneralizedMeanPoolingP()
                self.pool_list.append(pool)

        self.ASE_FC_list=nn.ModuleList()
        self.ASE_CONV_list=nn.ModuleList()
        for i in range(4):
            for j in range(4):
                fc_block=nn.Sequential(
                    nn.Linear(2048,1024),
                    nn.ReLU(),
                    nn.Linear(1024,2048),
                )
                conv_b=nn.Sequential(
                    nn.Conv2d(2048,1024,kernel_size=1),
                    nn.BatchNorm2d(1024),
                    nn.ReLU()
                )
                self.ASE_FC_list.append(fc_block)
                self.ASE_CONV_list.append(conv_b)

        self.fc_list=nn.ModuleList()
        for i in range(4):
            for j in range(4):
                fc=nn.Linear(1024,num_class)
                self.fc_list.append(fc)

    def ASE(self,x,index):
        feat=self.ASE_FC_list[index](x.view(x.shape[0],-1))
        x=x+torch.mul(x,F.sigmoid(feat).view(x.shape[0],-1,1,1))
        x=self.ASE_CONV_list[index](x)
        return x


    def forward(self,x,label):
        theta_list0 = []
        theta_list1 = []
        theta_list2 = []
        theta_list3 = []
        for i in range(self.N):
            zi = self.lambdak[:i].sum() / self.lambdak.sum()
            fi = (self.b0 - self.a0) * zi + self.a0
            theta_list0.append(fi)
        for i in range(self.N):
            zi = self.lambdak[:i].sum() / self.lambdak.sum()
            fi = (self.b1 - self.a1) * zi + self.a1
            theta_list1.append(fi)
        for i in range(self.N):
            zi = self.lambdak[:i].sum() / self.lambdak.sum()
            fi = (self.b2 - self.a2) * zi + self.a2
            theta_list2.append(fi)
        for i in range(self.N):
            zi = self.lambdak[:i].sum() / self.lambdak.sum()
            fi = (self.b3 - self.a3) * zi + self.a3
            theta_list3.append(fi)
        r_list = []
        for i in range(self.M):
            zi = self.lambdad[:i].sum()/self.lambdad.sum()
            fi = (self.R_max - self.R_min) * zi + self.R_min
            r_list.append(fi)
        theta_lists=torch.tensor([theta_list0, theta_list1, theta_list2, theta_list3])
        r_list = torch.tensor(r_list)
        self.grid = torch.zeros((4, 1, self.N, self.M, 2))
        for k in range(4):
            for i in range(self.N):
                for j in range(self.M):
                    self.grid[k,0,i,j,0]=r_list[j]*torch.sin(theta_lists[k,i])
                    self.grid[k,0,i,j,1]=r_list[j]*torch.cos(theta_lists[k,i])
        self.grid = self.grid.cuda()

        grid_features=[]
        for i in range(4):
            x=x.float()
            grid=self.grid[i]
            grid=grid.repeat(x.shape[0],1,1,1)
            grid_feature=F.grid_sample(x,grid)
            grid_features.append(grid_feature)
        loss=0
        logit_list=[]
        final_feature_list=[]
        #grid_features 4 x[0] 3 n m
        for i in range(4):
            features=self.backbone_list[i](grid_features[i])
            stride=int(features.shape[2]/4)
            for j in range(4):
                feature=features[:,:,j*stride:(j+1)*stride,:]
                feature=self.pool_list[i*4+j](feature)
                feature=self.ASE(feature,i*4+j)
                feature=feature.view(feature.shape[0],-1)
                logit=self.fc_list[i*4+j](feature)
                logit_list.append(logit)
                final_feature_list.append(feature)
        feat=torch.cat(final_feature_list,dim=1)
        if self.training:        
          for logit in logit_list:
              loss=loss+self.cls_criterion(logit,label)

          
          loss=loss+self.circle_criterion(feat,label)
        else:
          loss=0
        return feat,loss


In [60]:
def MARKET_EVAL_FUNC(dismat,q_pids,g_pids,q_camids,g_camids,max_rank=50):
    '''
    :param dismat: 每个query与其对应查询结果的距离
    :param q_pids: 每个query行人的id
    :param g_pids: 每个查询结果的行人的id
    :param q_camids: 摄像头id，对于同一个query，如果查询结果与query都来自一个摄像头，则丢弃
    :param g_camids: 同上
    :param max_rank:
    :return:
    '''
    num_q,num_g=dismat.shape
    if num_g<max_rank:
        max_rank=num_g

    ##根据相似度进行排序
    indices=np.argsort(dismat,axis=1)
    
    
    ##在排序结果找到同id图片，用于计算cmc
    matchs=(g_pids[indices]==q_pids[:,np.newaxis]).astype(np.int32)
    


    all_cmc=[]
    all_AP=[]
    num_valid_q=0

    for q_idx in range(num_q):
        q_pid=q_pids[q_idx]
        q_camid=q_camids[q_idx]

        '''当前query对应查询结果的排序'''
        order=indices[q_idx]

        '''若查询结果与query同摄像机且同id，删除'''
        remove=(g_pids[order]==q_pid)&(g_camids[order]==q_camid)
        keep=np.invert(remove)

        '''计算CMC'''

        '''首先先筛选'''
        orig_cmc=matchs[q_idx][keep]

        '''全是false'''
        if not np.any(orig_cmc):
            continue

        

        '''累加和，因为orig_cmc里面是true，false，true为1，得到cmc列表'''
        cmc=orig_cmc.cumsum()
        cmc[cmc>1]=1
        all_cmc.append(cmc[:max_rank])
        num_valid_q+=1 

        '''计算map'''

        '''预测正确的数量'''
        num_rel=orig_cmc.sum()

        '''累加和，用于计算'''
        tmp_cmc=orig_cmc.cumsum()

        '''计算准确率，前k个查询结果的准确率'''
        tmp_cmc=[x/(i+1) for i,x in enumerate(tmp_cmc)]

        '''计算召回率，召回率每次变化都是当查询结果id等于query时，故乘orig_cmc'''
        '''最终的结果类似 0,0,1,0,0,2,0,0,3这种，每个有值的都是查询正确的'''
        '''然后后面除以查询结果中同id数量，也就是上面num_rel就是找召回率'''
        '''但是这里其实已经提前乘上准确率了，最终结果类似于 0,0,1/3,0,0,2/6,0,0,3/9'''
        tmp_cmc=np.asarray(tmp_cmc)*orig_cmc

        '''以上为例，除以同id数量后，假设为五个'''
        '''0，0，1/3 /5，0，0，2/6 /5，0，0，3/9 /5'''
        '''等价于 0,0, 1/3(准确率)*1/5(召回率) ....'''
        
        if num_rel==0:
          AP=0
          print('AP=0')
        else:  
          AP = tmp_cmc.sum() / num_rel
          
        all_AP.append(AP)

    all_cmc = np.asarray(all_cmc).astype(np.float32)
    all_cmc = all_cmc.sum(0) / num_valid_q
    if len(all_AP)==0:
      
      return 0,0
    mAP = np.mean(all_AP)

    return all_cmc,mAP


class MARKET_MAP():
    def __init__(self,num_query,max_rank=50,feat_norm=True,one_day=True):
        '''

        :param num_query: 查询数量
        :param max_rank: 每个query限定的最大查询结果数量
        :param feat_norm:是否对特征进行l2norm
        '''
        self.num_query=num_query
        self.max_rank=max_rank
        self.feat_norm=feat_norm
        self.one_day=one_day
        
        self.reset()

    def reset(self):
        '''
        feats:模型返回的特征
        pids: 图片中行人的id
        camids:图片摄像头的id
        :return:
        '''
        self.feats=[]
        self.pids=[]
        self.camids=[]
        self.dates=[]

    def update(self,output):
        feat,pid,camid,date=output
    
        self.feats.append(feat)
        self.pids.append(pid)
        self.camids.append(camid)
        self.dates.append(date)

    def compute(self):
        feats=torch.stack(self.feats,dim=0)
          
        if self.feat_norm:
            feats=torch.nn.functional.normalize(feats,p=2)

        
        '''用于查询的图片的特征及其id和摄像头id'''
        
        qf=feats[:self.num_query]
        q_pids=np.asarray(self.pids[:self.num_query])
        q_camids = np.asarray(self.camids[:self.num_query])
        q_dates=np.asarray(self.dates[:self.num_query])  

        '''每个query查询结果的特征及其id和摄像头id'''
        gf = feats[self.num_query:]
        g_pids = np.asarray(self.pids[self.num_query:])
        g_camids = np.asarray(self.camids[self.num_query:])
        g_dates=np.asarray(self.dates[self.num_query:])
       
        if self.one_day:
            
            mean_map=0
            date=0
              
            q_index=np.where(q_dates==date)
            
            date_qf=qf[q_index]
            date_q_pids=q_pids[q_index]
            date_q_camids=q_camids[q_index]

            g_index=np.where(g_dates==date)
            date_gf=gf[g_index]
            date_g_pids=g_pids[g_index]
            date_g_camids=g_camids[g_index]
            m,n=date_qf.shape[0],date_gf.shape[0]
            
            if m==0:
              return 0
            distmat = torch.pow(date_qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(date_gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
            
            distmat.addmm_(1, -2, date_qf, date_gf.t())
            distmat = distmat.cpu().numpy()
            
            cmc,mAP=MARKET_EVAL_FUNC(distmat,date_q_pids,date_g_pids,date_q_camids,date_g_camids)
            mean_map+=mAP
            return cmc,mean_map
        else:
           
            mean_map=0
            
            date=0
            q_index=np.where(q_dates==date)
            
            date_qf=qf[q_index]
            date_q_pids=q_pids[q_index]
            date_q_camids=q_camids[q_index]

            g_index=np.where(g_dates!=date)
            date_gf=gf[g_index]
            date_g_pids=g_pids[g_index]
            date_g_camids=g_camids[g_index]
            m,n=date_qf.shape[0],date_gf.shape[0]
  
            if m==0 or n==0:
               return 0
            distmat = torch.pow(date_qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(date_gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
            
            distmat.addmm_(1, -2, date_qf, date_gf.t())
            distmat = distmat.cpu().numpy()
            
            cmc,mAP=MARKET_EVAL_FUNC(distmat,date_q_pids,date_g_pids,date_q_camids,date_g_camids)
            mean_map+=mAP
            return cmc,mean_map


In [61]:
class Checkpoint:
    def __init__(self, ckpt):
        self.ckpt = ckpt
        self.init = -1000

    def __call__(self, metric, model):
        if metric > self.init:
            self.init = metric
            torch.save({'model': model.state_dict()}, self.ckpt)

class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}



# NOTE(avik-pal): This code has been adapted from
#                 https://github.com/heykeetae/Self-Attention-GAN/blob/master/spectral.py
class SpectralNorm2d(nn.Module):
    r"""2D Spectral Norm Module as described in `"Spectral Normalization
    for Generative Adversarial Networks by Miyato et. al." <https://arxiv.org/abs/1802.05957>`_
    The spectral norm is computed using ``power iterations``.
    Computation Steps:
    .. math:: v_{t + 1} = \frac{W^T W v_t}{||W^T W v_t||} = \frac{(W^T W)^t v}{||(W^T W)^t v||}
    .. math:: u_{t + 1} = W v_t
    .. math:: v_{t + 1} = W^T u_{t + 1}
    .. math:: Norm(W) = ||W v|| = u^T W v
    .. math:: Output = \frac{W}{Norm(W)} = \frac{W}{u^T W v}
    Args:
        module (torch.nn.Module): The Module on which the Spectral Normalization needs to be
            applied.
        name (str, optional): The attribute of the ``module`` on which normalization needs to
            be performed.
        power_iterations (int, optional): Total number of iterations for the norm to converge.
            ``1`` is usually enough given the weights vary quite gradually.
    Example:
        .. code:: python
            >>> layer = SpectralNorm2d(Conv2d(3, 16, 1))
            >>> x = torch.rand(1, 3, 10, 10)
            >>> layer(x)
    """
    def __init__(self, module, name="weight", power_iterations=1):
        super(SpectralNorm2d, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        w = getattr(self.module, self.name)
        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]
        self.u = nn.Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        self.v = nn.Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        self.u.data = self._l2normalize(self.u.data)
        self.v.data = self._l2normalize(self.v.data)
        self.w_bar = nn.Parameter(w.data)
        del self.module._parameters[self.name]

    def _l2normalize(self, x, eps=1e-12):
        r"""Function to calculate the ``L2 Normalized`` form of a Tensor
        Args:
            x (torch.Tensor): Tensor which needs to be normalized.
            eps (float, optional): A small value needed to avoid infinite values.
        Returns:
            Normalized form of the tensor ``x``.
        """
        return x / (torch.norm(x) + eps)

    def forward(self, *args):
        r"""Computes the output of the ``module`` and appies spectral normalization to the
        ``name`` attribute of the ``module``.
        Returns:
            The output of the ``module``.
        """
        height = self.w_bar.data.shape[0]
        for _ in range(self.power_iterations):
            self.v.data = self._l2normalize(
                torch.mv(torch.t(self.w_bar.view(height, -1)), self.u)
            )
            self.u.data = self._l2normalize(
                torch.mv(self.w_bar.view(height, -1), self.v)
            )
        sigma = self.u.dot(self.w_bar.view(height, -1).mv(self.v))
        setattr(self.module, self.name, self.w_bar / sigma.expand_as(self.w_bar))
        return self.module.forward(*args)


In [62]:
from tqdm import tqdm
import logging
import albumentations as albu
import numpy as np
import torch
from torchtools.optim import RangerLars
from torch.cuda.amp import autocast,GradScaler

scaler=GradScaler()
class Trainer:
  def __init__(self,cfg,train_component):
    self.train_loader=train_component['train_loader']
    self.val_loader=train_component['val_loader']
    self.reid_model=train_component['model']
    self.optimizer=train_component['optimizer']
    self.scheduler=train_component['scheduler']
    self.logger=train_component['logger']
    self.num_class=train_component['num_class']
    self.num_query=train_component['num_query']
    self.checkpoint=train_component['checkpoint']
    
    if cfg['ema']:
      self.ema=train_component['ema']
   
    self.device=torch.device('cuda:0')
    self.reid_model=self.reid_model.to(self.device)
    self.cfg=cfg
    
    
  def train_one_epoch(self):
    face_id=0
   
    for images,sketchs,pids in self.train_loader:
      loss=0
      self.optimizer.zero_grad()
     
      
      sketchs=sketchs.to(self.device)
      pids=pids.to(self.device)
      with autocast():
        _,loss=self.reid_model(sketchs,pids)
        
      scaler.scale(loss).backward()
      scaler.step(self.optimizer)
      scaler.update() 
      if self.cfg['ema']:
        self.ema.update()
     
  
  
  def train(self):
        epochs=self.cfg['epochs']
        
        
        with tqdm(total=epochs) as pbar:
          for epoch in range(epochs):
              self.reid_model.train()
              self.train_one_epoch()
              print(1)
              if epoch%3==0:
                  if self.cfg['ema']:
                      self.ema.apply_shadow()
                  map0,map1,cmc0,cmc1=self.eval()
                  print('epoch:{}--map0:{}---map1:{}---cmc0:{}---cmc1:{}'.format(epoch,map0,map1,cmc0[0],cmc1[0]))
                  self.logger.info('epoch:{}--map0:{}---map1:{}'.format(epoch,map0,map1))
                  self.checkpoint((map0+map1)/2,self.reid_model)
                  if self.cfg['ema']:
                        self.ema.restore()    
              pbar.update(1)
     
        
  def eval(self):
    self.reid_model.eval()
    
     
    metric0=MARKET_MAP(self.num_query,one_day=True)
    metric1=MARKET_MAP(self.num_query,one_day=False)
    
    with torch.no_grad():
        for imgs,sketchs,pids,cams,dates in self.val_loader:
  
          sketchs=sketchs.to(self.device)  
          feat,_=self.reid_model(sketchs,pids)
          
          for out in zip(feat,pids,cams,dates):
            

            metric0.update(out)
            metric1.update(out)
            
    cmc0,map0=metric0.compute()
    cmc1,map1=metric1.compute()
    return map0,map1,cmc0,cmc1


In [64]:
import logging
def main(cfg):
  dataset=PRCC(cfg['root'])
  print(len(dataset.train))
  train_loader,val_loader,num_query,num_classes=make_dataloader(dataset,cfg)
  model=Mask_Branch(cfg,num_classes)
  if cfg['optimizer']=='adam':
    optimizer=torch.optim.Adam(model.parameters(),lr=cfg['lr'],weight_decay=cfg['weight_decay'])
  elif cfg['optimizer']=='sgd':
    optimizer=torch.optim.SGD(model.parameters(),lr=cfg['lr'],momentum=cfg['momentum'],weight_decay=cfg['weight_decay'])
  elif cfg['optimizer']=='rangerlars':
    optimizer=RangerLars(model.parameters(),lr=cfg['lr'])
  
  if cfg['scheduler']=='warmup':

    scheduler = WarmupMultiStepLR(optimizer,cfg['steps'], cfg['gamma'],
                  cfg['warmup_factors'],cfg['warmup_iters'], 
                  'linear')
  elif cfg['scheduler']=='flat':
    scheduler=Flat(optimizer,cfg,cfg['lr'])
  elif cfg['scheduler']=='cos':
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,5)
  
  ckpt=cfg['ckpt']
  logpt=cfg['logpt']
  logger=logging.getLogger()
  fh = logging.FileHandler("spam.log")
  fh.setLevel(logging.DEBUG)
  logger.addHandler(fh)
  checkpoint=Checkpoint(ckpt)
  train_component={}
  train_component['train_loader']=train_loader
  train_component['val_loader']=val_loader
  train_component['model']=model
  train_component['optimizer']=optimizer
  train_component['scheduler']=scheduler
  train_component['logger']=logger
  train_component['num_class']=num_classes
  train_component['num_query']=num_query
  train_component['checkpoint']=checkpoint
  trainer=Trainer(cfg,train_component)
  trainer.train()
  return trainer


In [ ]:
import random
import numpy as np
import torch
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.benchmark=True
torch.manual_seed(42)
torch.cuda.manual_seed(42)       
cfg={

 
    'num_workers':8,
    'train_bs':64,
    'test_bs':64,
    'train_K_instances':16,
     
    'epochs':20,
    'ckpt':'reid_test1.pt',
    'logpt':'test1.log',
   
    'momentum':0.9,
    'weight_decay':5e-4,
     
    'margin':0.3,
    'steps':[35,55],
    'warmup_iters':10,
    'warmup_factors':1/3,
    'gamma':0.1,
    'backbone':'eff_b0',
    'optimizer':'rangerlars',
    'scheduler':'cos',
   
    'ema':False,
    'N':128,
    'M':128,
    'lr':1e-3,
    'root':'/kaggle/working/datasets/prcc_proprecess'
}   

trainer=main(cfg)

In [ ]:
#test
map0,map1,cmc0,cmc1 = trainer.eval()
print("Between camera A and B(without clothes changing):")
print("Rank1 Acc:%.2f\nRank10 Acc:%.2f\nRank20 Acc:%.2f"%(cmc0[0],cmc0[9],cmc0[19]))
print("Between camera A and C(with clothes changing):")
print("Rank1 Acc:%.2f\nRank10 Acc:%.2f\nRank20 Acc:%.2f"%(cmc1[0],cmc1[9],cmc1[19]))